In [1]:
import numpy as np
import json
import os
import ROOT
from collections import OrderedDict

%jsroot on

Welcome to JupyROOT 6.18/00


# Draw single plots from json

The next cell just opens the content from a json file, creates TH1F out of them and draw histograms

In [2]:
def jsonToTH1( jsonFile, debug=False ):
    
    ## opening the json file
    with open(jsonFile) as json_file:
        data = json.load(json_file)
        
    ## priting the list of histograms in json
    if debug:
        print("In jsonFile: ", jsonFile, "the histograms found are: ")
        for i in data.keys(): print(i)

    ## initializing dictionaries for histograms (jupyter needs canvas AND histos)
    histoDict = OrderedDict()
        
    ## creating histograms with the information in json
    for var in data.keys():
        histoDict[var] = ROOT.TH1F( var+jsonFile, var, len(data[var]["edges"])-1, data[var]["edges"][0], data[var]["edges"][-1] )
        for icont in range(len(data[var]["contents"])):
            histoDict[var].SetBinContent( icont+1, data[var]["contents"][icont] )
            histoDict[var].SetBinError( icont+1, data[var]["contents_w2"][icont] )
        
    return histoDict

In [3]:
### Creating all the histograms
hDictRunAna = hDictMyRunAna = {}
hDictRunAna = jsonToTH1( "out_ttHTobb_M125_TuneCP5_13TeV-powheg-pythia8_runanalysis_resolved.json", debug=False )
hDictMyRunAna = jsonToTH1( "out_ttHTobb_M125_TuneCP5_13TeV-powheg-pythia8_myrunanalysis.json", debug=False )

In [7]:
canDict = {}
for var in hDictMyRunAna:
    if not 'boosted' in var: continue
    #if not 'nleps' in var: continue
    canDict[var] = ROOT.TCanvas("can"+var, "can"+var, 800, 500)
    canDict[var].Draw()
    #hDictRunAna[var].SetLineColor(ROOT.kRed)
    #hDictRunAna[var].SetMaximum( maxY*2)
    hDictMyRunAna[var].Draw()    
    canDict[var].Modified()
    canDict[var].Update()

In [6]:
#varList = [ "hist_ttHTobb_boosted_higgs_pt", "hist_ttHTobb_boosted_nbtags" ]

canDict = {}
for var in hDictMyRunAna:
    if not 'resolved' in var: continue
    if 'nelectrons' in var: continue
    if 'nmuons' in var: continue
    canDict[var] = ROOT.TCanvas("can"+var, "can"+var, 800, 500)
    canDict[var].Draw()
    maxY = max(hDictRunAna[var].GetMaximum(), hDictMyRunAna[var].GetMaximum())
    hDictRunAna[var].SetLineColor(ROOT.kRed)
    hDictRunAna[var].SetMaximum( maxY*2)
    hDictRunAna[var].Draw()
    hDictMyRunAna[var].Draw("histe same")
    #hDictPriv[var].Draw("hist")
    
    canDict[var].Modified()
    canDict[var].Update()


# Calculation of stat. only result

In [10]:
def likelihood(mu, data_i, s_i, b_i):
    b_i[b_i < 0] = 0.0
    s_i[s_i < 0] = 0.0

    sel = ((s_i>0) & (b_i>0))
    
    ret = data_i[sel] * np.log(np.full(data_i[sel].shape[0], mu)*s_i[sel] + b_i[sel]) - (mu*s_i[sel] + b_i[sel])
    #ret = data_i * np.log(np.full(data_i.shape[0], mu)*s_i + b_i) - (mu*s_i + b_i)
    return -np.sum(ret)

In [11]:
def make_fake_data(data, var, processes, cat):
    
    # calculate total MC prediction
    for p in processes:
        if processes.index(p) == 0:
            counts = data["hist_" + p + "_" + cat + "_" + var]["contents"]
        else:
            counts = [x + y for x, y in zip(counts, data["hist_" + p + "_" + cat + "_" + var]["contents"])]
    
    return counts

In [12]:
def stat_fit(data, var, processes, cat):
    
    fake_data = np.asarray(make_fake_data(data, var, processes, cat))
    sigs = ["ttHTobb", "ttHToNonbb"]
    for p in sigs:
        if sigs.index(p) == 0:
            sig_tot = data["hist_" + p + "_" + cat + "_" + var]["contents"]
        else:
            sig_tot = [x + y for x, y in zip(sig_tot, data["hist_" + p + "_" + cat + "_" + var]["contents"])]
    sig = np.asarray(sig_tot)
    print(sig.shape)
    
    bkgs = ["ttlf", "ttcc", "ttb", "tt2b", "ttbb"]
    for p in bkgs:
        if bkgs.index(p) == 0:
            bkg_tot = data["hist_" + p + "_" + cat + "_" + var]["contents"]
        else:
            bkg_tot = [x + y for x, y in zip(bkg_tot, data["hist_" + p + "_" + cat + "_" + var]["contents"])]
    bkg_tot = np.asarray(bkg_tot)
    print(bkg_tot.shape)
    print(fake_data.shape)
    
    import scipy.optimize as optimize
    mu0 = 0.0
    res = optimize.minimize(likelihood, mu0, (fake_data, sig, bkg_tot), bounds = [(None,None)])

    # Error of estimator
    err = lambda mu: likelihood(mu, fake_data, sig, bkg_tot)-(likelihood(res.x, fake_data, sig, bkg_tot)+0.5)

    down = res.x - optimize.fsolve(err,(res.x - 3.0))[0]
    up = optimize.fsolve(err,(res.x + 0.01))[0] - res.x

    print("Best fit:", res.x[0], "-", down[0], "+", up[0])
    

In [13]:
cat = "sl_jge4_tge2"
stat_fit(mc, "DNN_ROC", processes, cat)

KeyError: 'hist_ttHTobb_sl_jge4_tge2_DNN_ROC'

# ROC curve

In [ ]:
def ROC(data, processes, cat, var):
    
    sigs = ["ttHTobb", "ttHToNonbb"]
    for p in sigs:
        if sigs.index(p) == 0:
            sig_tot = data["hist_" + p + "_" + cat + "_" + var]["contents"]
        else:
            sig_tot = [x + y for x, y in zip(sig_tot, data["hist_" + p + "_" + cat + "_" + var]["contents"])]
    sig = np.asarray(sig_tot)
    
    bkgs = ["ttlf", "ttcc", "ttb", "tt2b", "ttbb"]
    for p in bkgs:
        if bkgs.index(p) == 0:
            bkg_tot = data["hist_" + p + "_" + cat + "_" + var]["contents"]
        else:
            bkg_tot = [x + y for x, y in zip(bkg_tot, data["hist_" + p + "_" + cat + "_" + var]["contents"])]
    bkg_tot = np.asarray(bkg_tot)
    
    nsig = sum(sig_tot)
    nbkg = sum(bkg_tot)
    bkg_tot = [x/nbkg for x in bkg_tot]
    sig_tot = [x/nsig for x in sig_tot]

    fpr = np.cumsum(bkg_tot)
    tpr = np.cumsum(sig_tot)
    
    from sklearn.metrics import auc
    roc_auc = auc(fpr, tpr, reorder=True)
    if roc_auc < 0.5:
        roc_auc = 1 - roc_auc
    
    fig = plt.figure(figsize=(6,6))
    plt.plot([0, 1], [0, 1], color='k', linestyle='--')
    plt.plot(tpr, fpr, label='ROC curve (area = %0.2f)' % roc_auc)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.0])
    plt.xlabel('False Positive Rate', fontsize=16)
    plt.ylabel('True Positive Rate', fontsize=16)
    plt.legend(loc="lower right", fontsize=16)
    
    return roc_auc
    

In [ ]:
cat = "sl_jge6_tge3"
ROC(mc, processes, cat, "DNN_ROC")

In [ ]:
roc = 0
for p in ["ttH", "ttbb", "tt2b", "ttb", "ttcc", "ttlf"]:
    roc += ROC(mc, processes, cat, "DNN_ROC_" + p)
print(roc/6.)

# Plot confusion matrix

In [ ]:
def calculate_confusion_matrix(data, process):
    pred = []
    for true_p in ["ttHTobb", "ttbb", "tt2b","ttb", "ttcc", "ttlf"]:
        ntot = sum(data["hist_" + true_p + "_" + cat + "_nleps"]["contents"])
        for pred_p in ["ttH", "ttbb", "tt2b", "ttb", "ttcc", "ttlf"]:
            n_pred = sum(data["hist_" + true_p + "_" + cat + "_DNN_ROC_" + pred_p]["contents"])/ntot
            pred.append(n_pred)
                   
    confusion_matrix = np.asarray(pred)
    cm = confusion_matrix.reshape((6,6))   
    
    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap = plt.cm.Blues)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=["ttHbb", "ttbb", "tt2b", "ttb", "ttcc", "ttlf"], yticklabels=["ttHbb", "ttbb", "tt2b", "ttb", "ttcc", "ttlf"],
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax

In [ ]:
cat = "sl_j4_tge3"
calculate_confusion_matrix(mc, processes)